In [1]:
import torch
from functions import get_loader, plot_histories, plot_history
from ae_functions import train_autoencoder, validate_cae, visualize_latent_space, visualize_reconstruction, get_latent_dataset
from ae_models import DeepCAE, DeepBatchCAE, SimpleCAE, SimpleBatchCAE, SimpleCAE_Dropout, SimpleBatchDropoutCAE
import torch.nn as nn
from mlp_functions import train_mlp, validate_mlp, decision_level_fusion
from mlp_models import rawMLP, MLP, dropout_MLP, ImprovedMLP
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [2]:
##### CONFIG
NUM_EPOCHS = 5
LR_RATE = 1e-3
BATCH_SIZE = 8
WEIGHT_DECAY = 1e-5

DEVICE = ""
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print("CUDA is available. Using GPU...")
else:
    DEVICE = torch.device("cpu")
    print("CUDA is not available. Using CPU...")

BASE_PATH = 'C:/Users/Frank/OneDrive/Bureaublad/ARC/deep-multimodal-learning/fusion'

# Define the tool names and actions
TOOL_NAMES = ['hook', 'ruler', 'spatula', 'sshot']
ACTIONS = ['left_to_right', 'pull', 'push', 'right_to_left']

# All available object names
OBJECTS = ['0_woodenCube', '1_pearToy', '2_yogurtYellowbottle', '3_cowToy', '4_tennisBallYellowGreen',
            '5_blackCoinbag', '6_lemonSodaCan', '7_peperoneGreenToy', '8_boxEgg','9_pumpkinToy',
            '10_tomatoCan', '11_boxMilk', '12_containerNuts', '13_cornCob', '14_yellowFruitToy',
            '15_bottleNailPolisher', '16_boxRealSense', '17_clampOrange', '18_greenRectangleToy', '19_ketchupToy']

sensor_color = "color"
sensor_left = "icub_left"
sensor_right = "icub_right"
sensor_depth = "depthcolormap"

CUDA is available. Using GPU...


In [3]:
train_loader = get_loader(BASE_PATH, OBJECTS, TOOL_NAMES, ACTIONS, sensor_left, "training", batch_size=BATCH_SIZE)
val_loader = get_loader(BASE_PATH, OBJECTS, TOOL_NAMES, ACTIONS, sensor_left, "validation", batch_size=BATCH_SIZE)
test_loader = get_loader(BASE_PATH, OBJECTS, TOOL_NAMES, ACTIONS, sensor_left, "testing", batch_size=BATCH_SIZE)

In [13]:
# cae_lossfunction = nn.MSELoss()
cae_lossfunction = nn.BCELoss()

#cae = SimpleCAE(input_channels=3).to(DEVICE)
cae = SimpleBatchCAE(input_channels=3).to(DEVICE)
#cae = SimpleCAE_Dropout(input_channels=3).to(DEVICE)
#cae = SimpleBatchDropoutCAE(input_channels=3, dropout_rate=0.25).to(DEVICE)
#cae = DeepCAE(input_channels=3).to(DEVICE)
#cae = DeepBatchCAE(input_channels=3).to(DEVICE)

optimizer= torch.optim.Adam(cae.parameters(), lr=LR_RATE)
# optimizer = torch.optim.SGD(cae.parameters(), lr=LR_RATE, momentum=0.9)
# optimizer = torch.optim.AdamW(cae.parameters(), lr=LR_RATE, weight_decay=1e-2)

trained_cae, cae_epoch_loss_history = train_autoencoder(cae,\
                                                        train_loader,\
                                                        cae_lossfunction,\
                                                        optimizer,\
                                                        is_depth=False,\
                                                        num_epochs=NUM_EPOCHS,\
                                                        device=DEVICE,\
                                                        visualize=False)

avg_val_loss, cae_validation_loss_history = validate_cae(trained_cae,\
                                                        test_loader,\
                                                        cae_lossfunction,\
                                                        is_depth = False,\
                                                        device = DEVICE)

Epoch [1/10], AVG Loss: 0.6351, AVG PSNR: 8.0480, AVG SSIM: 0.1510
Epoch [2/10], AVG Loss: 0.5375, AVG PSNR: 9.3953, AVG SSIM: 0.3520
Epoch [3/10], AVG Loss: 0.4707, AVG PSNR: 10.6804, AVG SSIM: 0.5259
Epoch [4/10], AVG Loss: 0.4185, AVG PSNR: 11.9835, AVG SSIM: 0.6602
Epoch [5/10], AVG Loss: 0.3770, AVG PSNR: 13.2819, AVG SSIM: 0.7566
Epoch [6/10], AVG Loss: 0.3437, AVG PSNR: 14.5655, AVG SSIM: 0.8246
Epoch [7/10], AVG Loss: 0.3169, AVG PSNR: 15.8367, AVG SSIM: 0.8727
Epoch [8/10], AVG Loss: 0.2953, AVG PSNR: 17.0757, AVG SSIM: 0.9064
Epoch [9/10], AVG Loss: 0.2777, AVG PSNR: 18.2766, AVG SSIM: 0.9303
Epoch [10/10], AVG Loss: 0.2630, AVG PSNR: 19.4788, AVG SSIM: 0.9478
Average Validation Loss: 0.2571732569485903
Average PSNR: 20.021577807683556
Average SSIM: 0.9542404986426908


In [49]:
# #plot_histories(cae_batch_loss_history, cae_validation_loss_history)
# visualize_reconstruction(trained_cae, test_loader, num_samples=2)
# visualize_latent_space(trained_cae, test_loader, n_components=2)
# # plot_history(mlp_epoch_avg_losses)
# # plot_history(mlp_epoch_accuracy)

In [14]:
model_path = "C:/Users/Frank/OneDrive/Bureaublad/ARC/deep-multimodal-learning/weights_ae/"
weight_name = "batchnorm/batch_cae_ne5_b8_depth.pth"
torch.save(trained_cae.state_dict(), model_path+weight_name)

In [4]:
model_path = "C:/Users/Frank/OneDrive/Bureaublad/ARC/deep-multimodal-learning/weights_ae/"
weight_name = "batchnorm/batch_cae_ne5_b8_color.pth"
trained_cae = SimpleBatchCAE().to(DEVICE)
trained_cae.load_state_dict(torch.load(model_path+weight_name))

<All keys matched successfully>

In [16]:
# Config MLP
mlp_lossfunction = nn.CrossEntropyLoss()  # Loss function
output_dim = 4 
NUM_EPOCHS = 5

# Extract features from the train and validation sets
train_dataset = get_latent_dataset(trained_cae, test_loader, add_noise=False, is_depth=False, device=DEVICE)
val_dataset = get_latent_dataset(trained_cae, val_loader, add_noise=False, is_depth=False, device=DEVICE)

# Create DataLoaders for the extracted features
mlp_train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
mlp_val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Initialize
input_dim = train_dataset[:][0].size(1)
print(input_dim)
# mlp = MLP(input_dim, output_dim).to(DEVICE)
# # mlp = ImprovedMLP(input_dim, output_dim).to(DEVICE)
# # #mlp = MLP_Dropout(input_dim, output_dim).to(DEVICE)
# # #mlp = rawMLP(input_dim, output_dim).to(DEVICE)
# mlp_optimizer = torch.optim.Adam(mlp.parameters(), lr=LR_RATE)

# # # Train the model
# trained_mlp = train_mlp(mlp, mlp_lossfunction, mlp_optimizer, mlp_train_loader, NUM_EPOCHS, DEVICE)

# # Validate the model
# validate_mlp(trained_mlp, mlp_lossfunction, mlp_val_loader, DEVICE)

147456


In [16]:
model_path = "C:/Users/Frank/OneDrive/Bureaublad/ARC/deep-multimodal-learning/weights_mlp/"
weight_name = "batchnorm/mlp_ne5_b8_depth_broken.pth"
torch.save(trained_mlp.state_dict(), model_path+weight_name)

In [10]:
# Initialize a new MLP model with the same architecture
mlp_loaded = MLP(input_dim, output_dim)  # Replace 'input_dim' and 'output_dim' with appropriate values
mlp_loaded.load_state_dict(torch.load(model_path+weight_name))
mlp_loaded = mlp_loaded.to('cuda')